# n-gram 언어 모델링

In [1]:
import nltk
from nltk.util import ngrams
from collections import Counter

In [2]:
text = "오늘은 날씨가 좋다. 오늘은 기분이 좋다. 오늘은 일이 많다. 오늘은 사람이 많다. 오늘은 날씨가 맑다."

In [ ]:
tokens = nltk.word_tokenize(text)   # 단어 단위로 토큰화

In [10]:
# 1-gram, 2-gram
unigram = tokens
bigram = list(ngrams(tokens, 2))

unigram_freq = Counter(unigram)
bigram_freq = Counter(bigram)
print(unigram_freq)
print(bigram_freq)

Counter({'오늘은': 5, '.': 5, '날씨가': 2, '좋다': 2, '많다': 2, '기분이': 1, '일이': 1, '사람이': 1, '맑다': 1})
Counter({('.', '오늘은'): 4, ('오늘은', '날씨가'): 2, ('좋다', '.'): 2, ('많다', '.'): 2, ('날씨가', '좋다'): 1, ('오늘은', '기분이'): 1, ('기분이', '좋다'): 1, ('오늘은', '일이'): 1, ('일이', '많다'): 1, ('오늘은', '사람이'): 1, ('사람이', '많다'): 1, ('날씨가', '맑다'): 1, ('맑다', '.'): 1})


In [ ]:
# Bigram 조건부 확률 계산 및 출력
for (w1, w2), freq in bigram_freq.items():  
    prob = freq / unigram_freq[w1]          # 조건부 확률 계산 (w1 뒤에 w2가 올 확률)
    print(f'P({w2}|{w1}) = {prob:.3f}')     # w1 다음에 w2가 등장할 확률

P(날씨가|오늘은) = 0.400
P(좋다|날씨가) = 0.500
P(.|좋다) = 1.000
P(오늘은|.) = 0.800
P(기분이|오늘은) = 0.200
P(좋다|기분이) = 1.000
P(일이|오늘은) = 0.200
P(많다|일이) = 1.000
P(.|많다) = 1.000
P(사람이|오늘은) = 0.200
P(많다|사람이) = 1.000
P(맑다|날씨가) = 0.500
P(.|맑다) = 1.000


### Perplexity 계산

In [ ]:
import math

# Perplexity의 평가 기준
# - 모델이 테스트 데이터에서 얼마나 적은 불확실성을 가지며 다음 단어를 잘 예측하는가

# 테스트 문장으로부터 바이그램 확률을 이용해 perplexity(혼란도)를 계산하는 함수
def compute_bigram_perplexity(test_text, unigram_freq, bigram_freq):
    test_tokens = nltk.word_tokenize(test_text) # 테스트 텍스트 토큰화
    test_bigrams = list(ngrams(test_tokens, 2)) # 테스트 문장 바이그램

    log_prob_sum = 0                            # 로그 확률 합 누적
    N = len(test_bigrams)                       # 평균용 바이그램 개수

    for bigram in test_bigrams:
        w1, w2 = bigram
        prob = bigram_freq.get(bigram, 0) / unigram_freq.get(w1, 1) # P(w2|w1) 계산
        if prob == 0:                           # 확률이 0이면
            prob = 1e-10                        # 매우 미세한 값으로 대체 (오류방지용)
        log_prob_sum += math.log2(prob)         # log2 확률로 누적(교차 엔트로피 계산용)

    cross_entropy = -log_prob_sum / N           # Cross-Entropy = -(1/N) * log_prob_sum
    perplexity = math.pow(2, cross_entropy)     # Perplexity = 2^(cross_entropy)

    return perplexity                         

   perplexity : 값이 작을수록 테스트 문장을 더 잘 예측한 모델이다. (1 = 혼동성 없음, 5 = 매 순간 평균 5개중에 혼동됨)

In [7]:
train_text = "자연어 처리는 재미있다. 자연어 처리는 어렵지만 도전하고 싶다. 오늘은 날씨가 좋다."

train_tokens = nltk.word_tokenize(train_text)

unigram = train_tokens
bigrams = list(ngrams(train_tokens, 2))

unigram_freq = Counter(unigram)
bigrams_freq = Counter(bigrams)

In [8]:
test_sentences = [
    "자연어 처리는 재미있다.",
    "자연어 처리는 어렵지만 도전하고 싶다.",
    "오늘은 날씨가 좋다.",
    "기계 번역은 어렵다.",
    "자연어 처리에 도전하고 싶다.",
    "오늘 날씨가 흐리다."
]

for sentence in test_sentences:
    pp = compute_bigram_perplexity(sentence, unigram_freq, bigrams_freq)
    print(sentence, "perplexity:", pp)

자연어 처리는 재미있다. perplexity: 1.2599210498948732
자연어 처리는 어렵지만 도전하고 싶다. perplexity: 1.148698354997035
오늘은 날씨가 좋다. perplexity: 1.0
기계 번역은 어렵다. perplexity: 10000000000.000008
자연어 처리에 도전하고 싶다. perplexity: 100000.00000000003
오늘 날씨가 흐리다. perplexity: 10000000000.000008


학습 문장에 나왔던 표현은 상대적으로 낮게 (1에 가깝게) 나오는 반면,  
학습에 없던 조합은 지표가 높게 나온다.